# FM 24 Moneyball

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import re

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show all columns in pandas
pd.set_option('display.max_columns', 500) 

%matplotlib inline

## Ingest and clean data

In [2]:
### Read raw data

# Select file path for raw data
path = r"C:\Users\kevmi\Documents\FM 24 Moneyball\FM Data Lab\Man Utd\FM 24 exports\Winter 2023\Defensive Midfielders.html"

# Ingest in a dataframe
df_players_raw = pd.read_html(path, encoding='utf-8')[0]

# Format column names 
df_players_raw.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_players_raw.columns]

# ---------------------------------------------------------------------------------------------------------------------

### Read processed Data lab data

# Select file path for processed data
path = r"C:\Users\kevmi\Documents\FM 24 Moneyball\FM Data Lab\Man Utd\FM Data Lab outputs\Winter 2023\Defensive Midfielders.csv"

# Ingest in a dataframe 
df_players = pd.read_csv(path)   

# Format column names 
df_players.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_players.columns]

# Bring ID from raw data
df_players = df_players.merge(df_players_raw[['name', 'age', 'height', 'weight', 'uid']], 
                 left_on=['name', 'age', 'height', 'weight'], 
                 right_on=['name', 'age', 'height', 'weight']).reset_index().rename(columns={'level_0': 'id'})

df_players['id'] = df_players['uid']

df_players.drop(columns='uid', inplace=True)

## Data cleaning

In [3]:
# Get transfer valuation estimation
def parse_transfer_value(val):
    if pd.isnull(val):
        return None

    # Remove dollar signs and spaces
    val = val.replace('$', '').replace(' ', '')

    # If it's a range like "850K-8.4M"
    if '-' in val:
        low_str, high_str = val.split('-')
        low = parse_single_value(low_str)
        high = parse_single_value(high_str)
        if low is not None and high is not None:
            return (low + high) * (6/10)
    else:
        return parse_single_value(val)

    return None  # fallback

def parse_single_value(s):
    try:
        if s.endswith('M'):
            return float(s[:-1]) * 1_000_000
        elif s.endswith('K'):
            return float(s[:-1]) * 1_000
        else:
            return float(s)  # just in case it's a plain number
    except:
        return None

# Apply to your DataFrame
df_players['price_estimate_M'] = df_players['transfer_value'].apply(parse_transfer_value) / 1000000

df_players.head()

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,chances_created/90,clear_cut_chances_created/90,pressures_attempted/90,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,tackles_attempted/90,tackles_completed/90,tackle_completion_%,tackles_failed/90,key_tackles/90,tackle_quality,interceptions/90,blocks/90,shots_blocked/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,aerial_challenges_attempted/90,duels_win_%,fouls_won/90,fouls_committed/90,clearances/90,distance_covered/90,attacking_actions/90,creative_actions/90,defensive_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,price_estimate_M
0,211232,Djéné Dakonam,"D (RC), DM",31,178 cm,71 kg,-,Getafe,LaLiga EA Sports,TOG (BEN),-,-,Scouting Required,"$20,500 p/w",$150K - $10.5M,-,Right,17,1496,6.56,0,88.00,0,43.38,38.08,88,4.03,9.29,0.48,1.11,0.48,1.11,0.12,0.12,8.18,2.47,30.15,11.37,6.98,4.39,62.89,3.73,3.07,82,0.66,0.24,2.07,1.93,0.78,0.42,3.79,2.35,61.90,1.44,38.10,0.24,3.79,72.00,0.24,1.80,1.56,12.57,0.59,0.78,16.00,2.29,9,100,98,6,6,2,5,3,7,4,6,7,59,87,22,6.390
1,211515,Baptiste Santamaria,"DM, M/AM (C)",28,183 cm,81 kg,-,Rennes,Ligue 1 Uber Eats,FRA,-,Resolute,Level-headed,"$16,000 p/w",$10M - $12.5M,-,Right,20,1741,6.87,1,82.90,1,73.56,66.07,90,8.27,11.24,1.03,1.41,0.98,1.34,0.07,0.05,12.25,3.31,27.00,8.22,9.15,-0.93,-10.16,4.03,2.95,73,1.09,0.00,2.36,1.71,0.47,0.26,3.67,1.09,29.58,2.58,70.42,0.10,3.67,52.35,0.47,1.71,0.72,13.79,1.12,1.19,11.02,3.36,47,100,38,63,47,28,18,40,29,32,19,6,31,26,20,13.500
2,219033,Alasana Manneh,"DM, M (C)",25,176 cm,72 kg,-,OB,3F Superliga,GAM,-,-,Scouting Required,"$5,500 p/w",$800K - $1.8M,-,Left,17,1507,6.98,0,88.65,2,81.16,73.64,91,6.81,8.39,1.07,1.32,0.90,1.10,0.42,0.42,9.91,2.33,23.49,6.99,10.51,-3.52,-33.49,2.39,1.43,60,0.96,0.06,3.08,1.37,0.42,0.30,1.85,0.42,22.58,1.43,77.42,0.00,1.85,43.66,1.02,0.78,0.90,12.94,1.07,1.67,7.62,2.75,87,100,7,67,85,70,58,77,47,68,50,55,2,2,76,1.560
3,12021720,Jean Michaël Séri,"DM, M (C)",32,168 cm,69 kg,-,Hull,Sky Bet Championship,CIV,-,-,Scouting Required,"$27,500 p/w",$325K - $3.2M,-,Right,15,1379,6.79,5,68.95,3,56.78,50.51,89,5.81,10.23,2.22,3.91,1.04,1.84,0.46,0.46,7.83,2.68,34.17,7.24,9.01,-1.77,-19.64,2.48,1.63,66,0.85,0.13,2.53,1.37,0.26,0.13,1.89,0.26,13.79,1.63,86.21,0.00,1.89,43.28,0.33,0.98,0.65,13.37,1.68,1.63,8.11,3.65,38,100,3,66,59,79,79,69,73,69,83,50,1,4,31,2.115
4,12080051,Pape Matar Sarr,"DM, M (C)",21,184 cm,70 kg,-,Tottenham,Premier League,SEN,-,Spirited,"Unflappable, Media-friendly","$100,000 p/w",$91M - $110M,-,Right,18,1524,6.94,0,84.67,4,65.79,57.64,88,8.33,12.66,1.48,2.24,1.48,2.24,1.56,0.59,11.16,3.43,30.69,8.68,9.33,-0.65,-6.97,3.13,2.72,87,0.41,0.06,2.94,2.07,0.30,0.12,4.55,1.65,36.36,2.89,63.64,0.30,4.55,56.92,0.41,1.06,0.65,14.00,2.20,2.42,13.25,4.67,65,100,51,90,72,81,84,62,79,59,88,57,61,62,57,120.600


## Plot function

In [5]:
def scatter_plot(df, x_metric, y_metric, z_metric, 
                 title, show_color_bar=False, show_name_label=True): 
    # Set text to 'name' if show_name_label is True, else empty string
    text_col = 'name' if show_name_label else None

    # Create plotly figure
    fig = px.scatter(
        df,
        x=x_metric,
        y=y_metric,
        color=z_metric,
        text=text_col,
        color_continuous_scale='RdYlGn',
        labels={z_metric: f'Color Scale ({z_metric})'},
        title=title,
        width=1200, 
        height=700    
    )

    # Format traces
    fig.update_traces(
        marker=dict(size=10), 
        textposition='top center',
        textfont=dict(size=10),
        customdata=df[['name', 'club']],  # Always pass 'name' for hovertemplate
        hovertemplate=(
            "<b>%{customdata[0]}</b><br>" +
            "<b>%{customdata[1]}</b><br><br>" +
            f"{x_metric}: " + "%{x}<br>" +
            f"{y_metric}: " + "%{y}<br>" +
            f"{z_metric}: " + "%{marker.color}<extra></extra>"
        )
    )

    fig.update_layout(
        title={
            'text': f'{title} ({len(df): ,.0f})',
            'x': 0.5,
            'xanchor': 'center'
        },
        coloraxis_colorbar=dict(title=z_metric),
        coloraxis_showscale=show_color_bar,  # Show or hide the color bar
        plot_bgcolor='white',
        paper_bgcolor='white',
        xaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False),
        yaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False)
    )

    fig.show()


## Passing analysis

### Full universe

In [6]:
# Initialize list of targets
targets = df_players['id']

scatter_plot(df_players,
             x_metric='passes_attempted/90', 
             y_metric='progressive_pass_rate',
             z_metric='pass_completion_%',
             title='Passing quality', 
             show_name_label=False
)

In [10]:
# Filter data 
mask = (
        (df_players['passes_attempted/90'] >= 40) & \
        (df_players['progressive_pass_rate'] >= 6) & \
        (df_players['pass_completion_%'] >= 85) & \
        (df_players['age'] <= 26) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='passes_attempted/90', 
             y_metric='progressive_pass_rate',
             z_metric='pass_completion_%',
             title='Passing quality', 
             show_name_label=False
)

# Keep only players that fulfill criteria
targets = list(df_targets['id'])

## Possession Analysis

In [9]:
# Full universe 
scatter_plot(df_targets,
             x_metric='possession_lost/90', 
             y_metric='possession_won/90',
             z_metric='poss+_/90',
             title='Possesion quality', 
             show_name_label=False
)

In [11]:
# Filter data 
mask = (
        (df_players['possession_lost/90'] <= 12) & \
        (df_players['possession_won/90'] >= 7) & \
        (df_players['poss+_/90'] >= 0) & \
        (df_players['age'] <= 26) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='possession_lost/90', 
             y_metric='possession_won/90',
             z_metric='poss+_/90',
             title='Possesion quality', 
             show_name_label=False
)

# Keep only players that fulfill criteria
targets = list(df_targets['id'])

## Aereal/Ground Dominance 

In [12]:
# Full universe 
scatter_plot(df_targets,
             x_metric='tackle_completion_%', 
             y_metric='headers_won_%',
             z_metric='duels_win_%',
             title='Dominance', 
             show_name_label=False
)

In [13]:
# Filter data 
mask = (
        (df_players['tackle_completion_%'] >= 65) & \
        (df_players['headers_won_%'] >= 45) & \
        (df_players['duels_win_%'] >= 60) & \
        (df_players['price_estimate_M'] <= 40) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='tackle_completion_%', 
             y_metric='headers_won_%',
             z_metric='duels_win_%',
             title='Dominance', 
             show_name_label=True
)

# Keep only players that fulfill criteria
targets = df_targets['id']

## Pressure quality

In [14]:
# Full universe 
scatter_plot(df_targets,
             x_metric='pressures_completed/90', 
             y_metric='pressure_success_%',
             z_metric='duels_win_%',
             title='Pressure quality', 
             show_name_label=True
)

In [16]:
# Filter data 
mask = (
        (df_players['pressures_completed/90'] >= 1.3) & \
        (df_players['pressure_success_%'] >= 25) & \
        (df_players['duels_win_%'] >= 65) & \
        (df_players['price_estimate_M'] <= 40) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='pressures_completed/90', 
             y_metric='pressure_success_%',
             z_metric='duels_win_%',
             title='Pressure quality', 
             show_name_label=True
)

# Keep only players that fulfill criteria
targets = df_targets['id']

## Presence

In [17]:
# Full universe 
scatter_plot(df_targets,
             x_metric='interceptions/90', 
             y_metric='blocks/90',
             z_metric='clearances/90',
             title='Presence', 
             show_name_label=True
) 

In [29]:
df_players[df_players['name'].str.contains('Wief')]

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,chances_created/90,clear_cut_chances_created/90,pressures_attempted/90,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,tackles_attempted/90,tackles_completed/90,tackle_completion_%,tackles_failed/90,key_tackles/90,tackle_quality,interceptions/90,blocks/90,shots_blocked/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,aerial_challenges_attempted/90,duels_win_%,fouls_won/90,fouls_committed/90,clearances/90,distance_covered/90,attacking_actions/90,creative_actions/90,defensive_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,price_estimate_M
309,37071145,Mats Wieffer,"D (C), DM, M (C)",24,188 cm,81 kg,Wnt,Feyenoord,Eredivisie,NED,-,Fairly Professional,Level-headed,"$8,500 p/w",$51M - $61M,-,Right,23,1802,7.23,0,78.35,309,68.42,61.28,90,7.19,10.51,1.85,2.7,1.75,2.55,0.7,0.4,10.44,2.85,27.27,10.19,9.44,0.75,7.94,2.55,2.15,84,0.4,0.05,4.25,2.15,0.15,0.0,6.19,3.7,59.68,2.5,40.32,0.4,6.19,66.86,0.75,0.6,0.3,13.56,3.03,2.45,13.87,6.94,100,100,50,94,97,99,99,97,98,97,98,93,83,74,84,67.2


In [24]:
# Filter data 
mask = (
        (df_players['interceptions/90'] >= 1.5) & \
        (df_players['blocks/90'] >= 0.4) & \
        (df_players['clearances/90'] >= 0.5) & \
        (df_players['price_estimate_M'] <= 40) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='interceptions/90', 
             y_metric='blocks/90',
             z_metric='clearances/90',
             title='Presence', 
             show_name_label=True
)

#df_targets = df_players[mask]

In [19]:
df_targets.groupby('division')['id'].count()

division
3F Superliga                            3
Bundesliga                              2
Campeonato Brasileiro Série A Assaí     4
Eredivisie                              3
LaLiga EA Sports                        3
Liga Portugal Betclic                   4
Liga Profesional de Fútbol              9
Ligue 1 Uber Eats                       3
Serie A TIM                             3
Sky Bet Championship                   11
Name: id, dtype: int64

In [21]:
df_players.groupby('division').get_group('Sky Bet Championship').sort_values('age')

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,chances_created/90,clear_cut_chances_created/90,pressures_attempted/90,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,tackles_attempted/90,tackles_completed/90,tackle_completion_%,tackles_failed/90,key_tackles/90,tackle_quality,interceptions/90,blocks/90,shots_blocked/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,aerial_challenges_attempted/90,duels_win_%,fouls_won/90,fouls_committed/90,clearances/90,distance_covered/90,attacking_actions/90,creative_actions/90,defensive_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,price_estimate_M
621,2000182795,Archie Gray,"DM, M (C)",17,182 cm,72 kg,-,Leeds,Sky Bet Championship,ENG (SCO),Trained in nation (15-21),-,Scouting Required,"$10,000 p/w",$15M - $19.5M,-,Right,8,858,7.10,12,42.90,621,67.97,60.63,89,6.08,8.95,1.47,2.16,1.36,2.01,0.23,0.21,11.33,3.36,29.63,9.86,9.65,0.21,2.18,3.15,2.52,80,0.63,0.42,4.29,2.41,0.63,0.42,2.83,1.57,55.56,1.26,44.44,0.00,2.83,68.42,0.84,0.73,1.15,14.15,2.54,1.89,15.52,5.45,69,100,96,91,89,87,81,87,88,95,83,96,21,86,28,20.700
623,2000189871,Lewis Miley,"DM, M/AM (C)",17,185 cm,72 kg,-,Leicester,Sky Bet Championship,ENG,Trained in nation (15-21),-,Scouting Required,"$25,000 p/w",$36M - $46M,-,Right,10,933,6.92,4,66.64,623,53.15,48.14,91,3.47,6.53,1.74,3.27,1.74,3.27,0.77,0.77,9.65,2.41,25.00,6.08,8.87,-2.79,-31.45,1.83,1.54,84,0.29,0.10,2.71,1.35,0.29,0.10,3.95,2.80,70.73,1.16,29.27,0.68,3.95,75.00,0.87,0.68,0.48,13.18,3.90,2.89,14.14,7.43,53,100,36,82,81,88,88,84,88,80,85,73,81,46,52,49.200
624,2000205927,Jobe Bellingham,"DM, M/AM (C), ST (C)",18,188 cm,80 kg,Inj,Sunderland,Sky Bet Championship,ENG (IRL),Trained in nation (15-21),-,Scouting Required,"$9,250 p/w",$8.4M - $11M,-,Right,19,1381,7.00,1,69.05,624,54.87,47.44,86,3.06,5.58,1.37,2.49,1.37,2.49,0.42,0.39,12.84,3.39,26.40,8.80,11.40,-2.60,-22.81,3.32,2.87,86,0.46,0.07,6.38,1.17,0.72,0.07,5.67,3.39,59.77,2.28,40.23,0.98,5.67,69.57,0.59,0.52,0.85,12.81,2.29,2.02,19.39,4.95,51,100,72,24,63,61,61,49,71,76,74,95,94,63,39,11.640
322,37085322,Lamare Bogarde,"D (LC), DM, M (C)",19,183 cm,76 kg,-,Preston,Sky Bet Championship,NED (SUR),Trained in nation (15-21),Balanced,Media-friendly,"$5,000 p/w",$10.5M - $15.5M,-,Right,16,1213,6.90,1,71.35,322,53.79,47.78,89,4.38,8.14,0.74,1.38,0.74,1.38,0.22,0.22,12.39,2.52,20.36,9.35,7.49,1.86,24.83,3.71,2.52,68,1.19,0.22,2.38,2.15,0.59,0.15,3.71,1.71,46.00,2.00,54.00,0.15,3.71,57.00,0.30,1.56,1.41,13.51,1.45,1.11,13.79,2.30,44,100,87,42,54,46,66,28,44,47,59,86,45,76,30,15.600
588,2000045039,Jordan James,"DM, M (C)",19,178 cm,64 kg,-,Birmingham,Sky Bet Championship,WAL (ENG),Trained in nation (15-21),-,Scouting Required,"$12,000 p/w",$8.4M - $11M,-,Right,6,854,6.82,14,42.70,588,42.68,38.04,89,1.79,4.20,0.95,2.22,0.95,2.22,0.59,0.53,11.49,2.53,22.02,7.80,8.22,-0.42,-5.11,3.06,2.21,72,0.84,0.11,2.23,1.90,0.42,0.21,4.43,1.90,42.86,2.53,57.14,0.63,4.43,54.93,0.84,1.37,0.95,13.46,2.44,1.58,15.24,4.53,47,100,56,31,54,75,83,61,71,70,78,94,77,73,53,11.640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,